In [1]:
!pip install pyspark

In [2]:
import pyspark
import numpy as np
from pyspark.sql import SparkSession
import time
import random
import os

# 0. Перевірка та ініціалізація Spark (важливо для VS Code)
# -----------------------------------------------------------

# Переконайтеся, що Spark і PySpark правильно налаштовані.
# Якщо ви використовуєте 'spark-submit' або інше середовище, 'local[*]' буде достатньо.
# У VS Code може знадобитися вказати SPARK_HOME, якщо він не налаштований.

# 1. Створення RDD та базові операції
# ------------------------------------

# Створення SparkSession та SparkContext
# appName: 'Lab3_RDD_Operations', master: 'local[*]'
try:
    spark = SparkSession.builder.appName("Lab3_RDD_Operations").master("local[*]").getOrCreate()
    sc = spark.sparkContext  # Отримання SparkContext
    print("1. SparkContext успішно створено.")
except Exception as e:
    print(f"Помилка ініціалізації Spark: {e}")
    print("Переконайтеся, що Apache Spark встановлено і налаштовано.")
    exit()

# Створення RDD зі списком слів (не менше 100 слів)
base_words = ["apple", "banana", "cherry", "date", "fig", "grape", "kiwi", "lemon", "lime", "mango", "orange", "papaya", "quince", "raspberry", "strawberry", "ugli", "vanilla", "watermelon"]
word_list = (base_words * 5) + ["blueberry", "blackberry", "apricot", "avocado"] * 5
word_list.extend(["Bumblebee", "Bat", "Ball", "Basket", "Book"] * 5)
random.shuffle(word_list) # Перемішування слів
print(f"   -> Створено список із {len(word_list)} слів.")

# Створення RDD
words_rdd = sc.parallelize(word_list)
print("   -> Приклад RDD слів:", words_rdd.take(5))

# -----------------------------------------------------------
print("\n" + "="*50)

# 2. Трансформації над RDD
# ------------------------

print(" 2. Трансформації над RDD")

# Трансформація map(): перетворення всіх рядків на великі літери
uppercase_rdd = words_rdd.map(lambda word: word.upper())
print("   -> Приклад після map() (великі літери):", uppercase_rdd.take(5))

# Трансформація filter(): отримання елементів, що починаються з літери "B"
b_words_rdd = uppercase_rdd.filter(lambda word: word.startswith("B"))
print(f"   -> Кількість слів на 'B' (до distinct): {b_words_rdd.count()}")
print("   -> Приклад після filter() (слова на 'B'):", b_words_rdd.take(5))

# Трансформація distinct(): видалення всіх дублікатів
distinct_b_words_rdd = b_words_rdd.distinct()
print(f"   -> Кількість слів на 'B' (після distinct): {distinct_b_words_rdd.count()}")
print("   -> Результат distinct():", distinct_b_words_rdd.collect())

# -----------------------------------------------------------
print("\n" + "="*50)

# 3. Робота з числовими даними
# ----------------------------

print(" 3. Робота з числовими даними")

# Створення RDD з числових даних (випадковий набір не менше 100 чисел)
numeric_data = [random.randint(1, 100) for _ in range(120)] # 120 чисел
numbers_rdd = sc.parallelize(numeric_data)
print("   -> Приклад RDD чисел:", numbers_rdd.take(5))

# Трансформація: кожне число піднести до квадрату
squared_rdd = numbers_rdd.map(lambda x: x * x)
print("   -> Приклад RDD після піднесення до квадрату:", squared_rdd.take(5))

# Трансформація: відібрати тільки парні числа (квадрати)
even_squared_rdd = squared_rdd.filter(lambda x: x % 2 == 0)
print("   -> Приклад RDD після фільтрації парних чисел:", even_squared_rdd.take(5))

# Дія (Action): підрахунок кількості парних чисел
even_count = even_squared_rdd.count()
print(f"   -> Підрахунок кількості парних чисел (дія count()): {even_count}")

# -----------------------------------------------------------
print("\n" + "="*50)

# 4. Агрегація даних
# ------------------

print(" 4. Агрегація даних")

# Знайти суму всіх елементів RDD (numbers_rdd)
# Дія reduce: сума всіх елементів
sum_of_numbers = numbers_rdd.reduce(lambda a, b: a + b)
print(f"   -> Сума всіх елементів (reduce): {sum_of_numbers}")

# Знайти добуток всіх елементів
# Використовуємо reduce. Обмежимося невеликою кількістю чисел, щоб уникнути переповнення int/long
product_rdd_small = sc.parallelize(numbers_rdd.take(10))
try:
    product_of_numbers = product_rdd_small.reduce(lambda a, b: a * b)
    print(f"   -> Добуток перших 10 елементів (reduce): {product_of_numbers}")
except Exception:
    # Якщо число занадто велике, виведемо його в науковому форматі після перетворення на float
    product_of_numbers_float = product_rdd_small.map(float).reduce(lambda a, b: a * b)
    print(f"   -> Добуток (float) перших 10 елементів: {product_of_numbers_float:.2e}")


# -----------------------------------------------------------
print("\n" + "="*50)

# 5. Кешування та повторне використання RDD
# -----------------------------------------

print(" 5. Кешування та повторне використання RDD")

# Використовуємо cache() для збереження RDD з квадратами чисел (squared_rdd)
squared_rdd.cache()
print("   -> Результати squared_rdd кешовано за допомогою cache().")

# Підрахунок кількості елементів у RDD кілька разів

# 1. Перше обчислення (виконує обчислення і записує до кешу)
start_time = time.time()
count1 = squared_rdd.count()
end_time = time.time()
time1 = end_time - start_time
print(f"   -> Підрахунок 1: Кількість = {count1}, Час = {time1:.6f} сек (Обчислення + Кешування)")

# 2. Друге обчислення (читання з кешу)
start_time = time.time()
count2 = squared_rdd.count()
end_time = time.time()
time2 = end_time - start_time
print(f"   -> Підрахунок 2: Кількість = {count2}, Час = {time2:.6f} сек (Читання з кешу)")

# 3. Третє обчислення (читання з кешу)
start_time = time.time()
count3 = squared_rdd.count()
end_time = time.time()
time3 = end_time - start_time
print(f"   -> Підрахунок 3: Кількість = {count3}, Час = {time3:.6f} сек (Читання з кешу)")

# Демонстрація переваги
print(f"\n   💡 Висновок: Час 2-го/3-го підрахунків ({time2:.6f} с / {time3:.6f} с) значно менший, ніж Час 1-го ({time1:.6f} с), що демонструє перевагу кешування.")

# -----------------------------------------------------------
print("\n" + "="*50)

# Зупинка SparkContext
sc.stop()
print("✅ SparkContext зупинено.")

1. SparkContext успішно створено.
   -> Створено список із 135 слів.
   -> Приклад RDD слів: ['ugli', 'kiwi', 'fig', 'quince', 'lime']

 2. Трансформації над RDD
   -> Приклад після map() (великі літери): ['UGLI', 'KIWI', 'FIG', 'QUINCE', 'LIME']
   -> Кількість слів на 'B' (до distinct): 40
   -> Приклад після filter() (слова на 'B'): ['BLUEBERRY', 'BUMBLEBEE', 'BLACKBERRY', 'BLACKBERRY', 'BAT']
   -> Кількість слів на 'B' (після distinct): 8
   -> Результат distinct(): ['BUMBLEBEE', 'BASKET', 'BOOK', 'BLUEBERRY', 'BLACKBERRY', 'BAT', 'BALL', 'BANANA']

 3. Робота з числовими даними
   -> Приклад RDD чисел: [43, 13, 76, 49, 39]
   -> Приклад RDD після піднесення до квадрату: [1849, 169, 5776, 2401, 1521]
   -> Приклад RDD після фільтрації парних чисел: [5776, 196, 6724, 4356, 4900]
   -> Підрахунок кількості парних чисел (дія count()): 60

 4. Агрегація даних
   -> Сума всіх елементів (reduce): 6480
   -> Добуток перших 10 елементів (reduce): 16897815745915104

 5. Кешування та повтор